# Read me
This template is meant to be a starter for your customized DREEM output data analysis.

- To install this library, please check the installation on the [Git repo](https://github.com/yvesmartindestaillades/NAP).
- To learn how to use this library, please get through the [tutorial](tutorial.ipynb).


# Turner overthrown

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from os.path import exists, dirname
import os, sys
import string
import numpy as np
import seaborn as sns

from dreem_nap import data_wrangler, database, plot, utils
from dreem_nap.study import Study


# Step 1: Data wrangling
### Step 1.1: Define your study and some basics about your project

In [ ]:
# Set your root folder for the database (at the moment, keep Yves)
folder = 'Yves'

# Select your study
study_name = 'temperature' 

## Set your base coverage high-pass filter value
min_bases_cov = 1000 

# Set the resolution for the plots
mpl.rcParams['figure.dpi'] = 200 # the highest the resolution, the slowest the plotting
mpl.rcParams["figure.figsize"] = [25, 7]
#plt.rcParams["figure.autolayout"] = True

# Depending on the study you select, you'll get a series of samples. You can also create new studies using this dictionary.
# Here's an example.
studies = data_wrangler.load_studies('../data/studies.csv')

study = studies.loc[study_name]
study

### Step 1.2: Process new pickle files and push them to Firebase
- Select which samples you want to push to Firebase.
To plot automatically arrays of samples, see [tutorial](tutorial.ipynb), section 3.2.
- Process samples and push them to Firebase.

In [ ]:
## Pickle files to process and to push to Firebase
pickles_list = [] # Can be samples if you want to process the samples from your study

pickles = {pickle: f"../data/FULLSET/{pickle}/mutation_histos.p" for pickle in pickles_list}

# Indicate the location of your RNA structure file
RNAstructureFile = '../data/RNAstructureFile.csv'

# Default location for your local database (JSON file)
json_file = '../data/db.json'

# If the user gives some new pickles files, push them to the firebase, then pull the entire firebase
if len(pickles):
    data_wrangler.push_samples_to_firebase(pickles = pickles,
                        RNAstructureFile = RNAstructureFile,
                        min_bases_cov = min_bases_cov, 
                        folder=folder)

### Step 1.3: Pull the data from the Firebase and clean/reformat it.
`df` is used for the analysis. Each of the construct have above 1000 reads for each sample.     
`df_full` is used for quality quality analysis. It has all constructs above 1000 valid reads for each sample individually.

In [4]:
# Pull the firebase
#df_database = database.load(samples=samples, username=username)

#data_wrangler.dump_dict_json(JSONFileDict=json_file, df=df_database)
df_database = data_wrangler.json_load(json_file)

# Clean and reformat the dataset
df, df_full = data_wrangler.clean_dataset(df_database=df_database,
                                             study=study)
print(f"{df.groupby('construct')['samples_covered'].count().count()} constructs have more than {min_bases_cov} reads for each base of their ROI on each sample")
        

43 constructs were dropped because deltaG was 'void'
108 constructs have more than 1000 reads for each base of their ROI on each sample


# Step 2: Data quality analysis

It's always hard to realize that you were analysing noise. Here, we'll get through a series a plot to check the data sanity.

### Get the list of samples and constructs:

In [5]:
print(f"samples are: {study.samples}")
print(f"constructs are: {df.construct.unique()}")

samples are: ['D7', 'E7', 'F7', 'G7', 'H7', 'A8', 'B8', 'C8']
constructs are: [  167   185   286   322   373   381   597   980  1009  1055  1063  1092
  1120  1495  1895  1994  2069  2093  2228  2323  2506  2524  2570  2780
  2797  3014  3163  3273  3337  3580  3708  3796  3925  3948  4274  4362
  4493  4585  4672  4708  4727  4758  5014  5269  5435  5440  5988  6317
  6458  6711  6896  7026  7172  7384  7487  7545  7808  7845  8106  8136
  8240  8280  8281  8351  8369  8422  8427  8430  8438  8574  8594  8695
  8758  9176  9211  9237  9302  9332  9572  9629  9796  9832  9843  9849
 10044 10162 10190 10663 10759 10803 10948 10992 11024 11035 11047 11197
 11217 11390 11499 11642 11659 11775 11825 12079 12108 12306 12361 12419]


### Explore the data
`get_roi_info(df=df, sample=sample, construct=construct)` gives information about the Region of Interest (ROI) of a (sample, construct) pair.

In [6]:
#sample, construct = utils.rand_sample_construct(df)
#utils.get_roi_info(df=df, sample=sample, construct=construct)#.xs((True, '0'),level=('paired','roi_structure_comparison'))   

#utils.columns_to_csv(df, study= study, columns=['sample','construct','full_sequence'],title='test.csv',path='')
samples = study.samples
df[df.samp.isin(samples)]

,samp,construct,cov_bases,cov_bases_roi,cov_bases_sec_half,data_type,del_bases,end,flank,full_deltaG,...,roi_end_index,roi_sequence,roi_start_index,roi_structure_comparison,skips_low_mapq,skips_short_read,skips_too_many_muts,start,sub-library,samples_covered
0,A8,167,"[0.0, 2671.0, 2669.0, 2683.0, 2675.0, 2570.0, ...",2656,1262,DMS,"[0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 0.0, 1.0, 0.0, ...",170,flank_1,-38.6,...,109,CCACCUGUAUAUAUCGGGUCCGAUAUAUACAGGUGG,73,000000000000000000000000000000000000,5012,0,3,1,structured PUM2 hairpin variants,8
1,A8,185,"[0.0, 27813.0, 27500.0, 27861.0, 27962.0, 2725...",24812,15600,DMS,"[0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 3.0, 0.0, ...",170,flank_3_partial,-63.0,...,114,UACAAAUCAGUGUAUAUAUGCCCCCAUAUAUACACUGAUUUGUA,70,00000000000000000000000000000000000000000000,17978,0,3,1,structured PUM2 hairpin variants,8
2,A8,286,"[0.0, 2851.0, 2847.0, 2842.0, 2853.0, 2739.0, ...",1724,559,DMS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",170,flank_1,-26.1,...,104,CUGUAUAUAUGCAAACAUAUAUACAG,78,00000000000000000000000000,2137,0,0,1,structured PUM2 hairpin variants,8
3,A8,322,"[0.0, 4323.0, 4323.0, 4319.0, 4331.0, 4255.0, ...",2044,319,DMS,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",170,flank_1,-26.6,...,108,GUCACGCAAAUGAGGAUCACCCAUGUGCGUGAC,75,000000000000000000000000000000000,5358,0,2,1,mismatched MS2 hairpin variants,8
4,A8,373,"[0.0, 12683.0, 12692.0, 12675.0, 12712.0, 1267...",3131,3,DMS,"[0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 1.0, 0.0, ...",170,flank_1,-14.8,...,101,UGUAUAUAGUAAUAUAUACA,81,10000000000000000001,12147,0,1,1,structured PUM2 hairpin variants,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,H7,12079,"[0.0, 38949.0, 38995.0, 38860.0, 38990.0, 3861...",4039,26,DMS,"[0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 6.0, 0.0, ...",170,flank_1,-17.8,...,113,UGUAUAUAUUUUCUUUUCUUUUCUUUUCUUUUCUUUAUAUACA,70,1000000000000000000000000000000000000000001,19914,0,0,1,structured PUM2 hairpin variants,8
1204,H7,12108,"[0.0, 25696.0, 25669.0, 25644.0, 25706.0, 2542...",2741,7,DMS,"[0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 3.0, 0.0, ...",170,flank_1,-17.3,...,111,UGUAUAAUAUACACAUGUAUAAUAUACACAUGUAUAAUAU,71,0000000000000000000000000000000000000000,24068,0,0,1,functional sites with multiple PUM Binding Sites,8
1205,H7,12306,"[0.0, 16871.0, 16846.0, 16875.0, 16910.0, 1667...",12203,322,DMS,"[0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 0.0, 2.0, 0.0,...",170,flank_1,-24.4,...,103,GCGAAUGAGGAUCACCCACGUGC,80,01011101001011111101001,12215,0,1,1,mismatched MS2 hairpin variants,8
1206,H7,12361,"[0.0, 19807.0, 19761.0, 19817.0, 19814.0, 1941...",10672,1395,DMS,"[0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 0.0, 0.0, 0.0, ...",170,flank_1,-24.5,...,108,CUGUAUAUAUGUUUUCUUUUCUUCAUAUAUACAG,74,0000000000000000000000000000000000,11006,0,2,1,structured PUM2 hairpin variants,8


### (sample, construct)-specific base coverage plot

In [ ]:
samp, construct = utils.rand_sample_construct(df)
plot.base_coverage(df, samp, construct)

### Plot the base coverage per construct distribution

In [ ]:
plot.base_coverage_ROI_for_all_constructs(df=df_full)

### Sanity-check construct-wise base coverage plots
Plot randomly picked sequences to check the quality of the data.

In [ ]:
plot.random_9_base_coverage(df=df)

### Heatmap of the roi part coverage

In [ ]:
plot.heatmap(df = df, 
             column="cov_bases_roi")

### Heatmap of the second half coverage

In [ ]:
plot.heatmap(df = df, 
                column="cov_bases_sec_half")

# Step 3: Data analysis
In this part, we know that we read good data, and we want to visualize it through different plots.

### Analysis parameters

In [ ]:
# Display the plots on this notebook? Not recommended if numerous plots
show_plots = True

# Constructs used
a_few_constructs = df.construct.unique()[:3].tolist()
first_construct = df.construct.unique()[0].tolist()
constructs_per_name = {
    'all_constructs': df.construct.unique().tolist(),
    str(a_few_constructs) : a_few_constructs,
    str(first_construct): [first_construct]
}

# Select constructs here
constructs_name = str(a_few_constructs)

# Define what you will analyse
constructs = constructs_per_name[constructs_name]

### Big script to run every selected function

In [ ]:
# Analysis run in this script
analysis = {'base_per_base_partition':False,
            'base_per_base_sequence': True,
            'deltaG': True,
            'sample_comparison':False,
            'columns_csv': True,
            'deltaG_construct': True
            }

# Write here a script to get your plots 

### Mutation sequence-wise

`plot.mutation_rate(df, sample, construct, plot_type, index, normalize)` plots the mutation rate base-wise for a given construct of a given sample as a barplot. 
Arguments:
- `plot_type` :
    - `'sequence'` : each bar is colored w.r.t to the base of the original sequence.
    - `'partition'` : each bar shows the partition of into which bases this base mutates.
- `index`:
    - `'index'`: each base is identified with its position number
    - `'base'`: each base is identified with its type (A, C, G, T)

In [ ]:
for construct in constructs:
    for samp in samples:
        plot.mutation_rate(df=df,
                           sample=samp,
                           construct=construct,
                           plot_type='sequence',
                           index='index')
        utils.save_fig(path=f"data/figs/date/{study}/mut_per_base/sequence/{construct}/", 
                    title=f"base_per_base_sequence_{samp}_{construct}")
        plt.close(not show_plots)

### DeltaG plots

In [ ]:
for samp in samples:
    plot.deltaG(df=df, sample=samp)

    utils.save_fig(path=f"data/figs/date/{study}/deltaG/", 
             title=f"deltaG_{samp}")

    plt.close(not show_plots)

### samples correlation

In [ ]:
for construct in [constructs[0]]:
        df_global_corr = plot.correlation_n_samples(df, samples, construct)
        utils.save_fig(path=f"data/figs/date/correlation/{study}", 
                      title=f"correlation_{study}_{construct}")
        plt.title(f"Study: {study}")
        plt.close(not show_plots)
        print(construct, end=' ')

In [ ]:
show_plots = False
for study in studies:
    samples = study['samples']
    for constructs in df.construct.unique():
        constructs_name = constructs
        df_global_corr = plot.correlation_n_samples(df, samples, constructs)
        plt.title(f"Correlation between samples for study: {study}, constructs: {constructs_name}")
        utils.save_fig(path=f"data/figs/date/correlation/{study}/{constructs_name}", 
                        title=f"correlation_fit_{study}_{constructs_name}")
        plt.close(not show_plots)

        for plt_type in ['r_value', 'slope']:
            pivot = df_global_corr.pivot("sample_0","sample_1", plt_type).astype(float)
            f, ax = plt.subplots(figsize=(28, 10))
            sns.heatmap(pivot, annot=False, linewidths=0, ax=ax)#, norm=LinNorm())
            plt.title(f"{plt_type} of the correlation between samples for study: {study}, constructs: {constructs_name}")
            utils.save_fig(path=f"data/figs/date/correlation/{study}/{constructs_name}", 
                            title=f"correlation_{plt_type}_{study}_{constructs_name}")
            plt.close(not show_plots)

### Heat Map

In [ ]:
#pivot = df_global_corr.pivot("sample_0","sample_1", 'r_value').astype(float)
df_global_corr

In [ ]:
for plt_type in ['r_value', 'slope']:
    pivot = df_global_corr.pivot("sample_0","sample_1", plt_type).astype(float)
    f, ax = plt.subplots(figsize=(28, 10))
    sns.heatmap(pivot, annot=False, linewidths=0, ax=ax)#, norm=LinNorm())
    plt.title(plt_type)
    utils.save_fig(path=f"data/figs/date/global_correlation/{study}", 
                    title=f"correlation_{plt_type}_{study}_all_constructs")
   # plt.close(not show_plots)

### Global correlation plot

In [ ]:
heat_map = True

for study in studies:
    samples = study['samples']
    df_global_corr = plot.correlation_n_samples(df, samples, constructs)
    utils.save_fig(path=f"data/figs/date/global_correlation/{study}", 
                    title=f"correlation_{study}_all_constructs")
    plt.close(not show_plots)

    if heat_map:
        for plt_type in ['r_value', 'slope']:
            pivot = df_global_corr.pivot("sample_0","sample_1", plt_type).astype(float)
            f, ax = plt.subplots(figsize=(28, 10))
            sns.heatmap(pivot, annot=False, linewidths=0, ax=ax, norm=LogNorm())
            utils.save_fig(path=f"data/figs/date/global_correlation/{study}", 
                            title=f"correlation_{plt_type}_{study}_all_constructs")
            plt.close(not show_plots)

In [ ]:
show_plots = 1
for study in studies:
    samples = studies[study]['samples']
    if study == 'all_samples': continue
    plot.correlation_n_samples(df, samples, constructs)
    plot.save_fig(path=f"data/figs/date/global_correlation/{study}", 
                title=f"correlation_{study}_all_constructs")
    plt.close(not show_plots)

 ### Conditions vs mutations #TODO

In [ ]:
def conditions_vs_mutations(df, samples, study, ):
    df_use = df.set_index(['samp','construct'])

    fig = plot.define_figure(title=samp,
                            xlabel='deltaG [kcal]',
                            ylabel='Mutation ratio',
                            figsize=(20,5))

    stack_for_plot = {'0':{'x':[],'y':[]},'1':{'x':[],'y':[]}}

    for construct in df.construct.unique():
        roi_part = utils.get_roi_info(df=df, sample=samp, construct=construct)
        for base in ['A','C']:
            for roi_struct_comp in ['0','1']:
                try:    
                    this_base_mut =  roi_part.xs((base,True,roi_struct_comp), level=('base','paired','roi_structure_comparison'))
                    stack_for_plot[roi_struct_comp]['x'].extend(this_base_mut['roi_deltaG'].to_list())
                    stack_for_plot[roi_struct_comp]['y'].extend(this_base_mut['mut_rate'].to_list())
                except:
                    continue
    plt.plot(stack_for_plot['0']['x'],stack_for_plot['0']['y'],'b.')
    plt.plot(stack_for_plot['1']['x'],stack_for_plot['1']['y'],'r.')
    plot.fit_deltaG(df_use, samp)
    plt.legend(['A and C bases of the ROI, predicted paired by RNAstructure for both the ROI sequence and the full sequence',\
                'A and C bases of the ROI part, predicted paired by RNAstructure for the full sequence but not for the ROI sequence'])
    plt.ylim([0,0.15])
    fig.tight_layout()

### Temperature vs reactivity plots

In [ ]:
plot.mut_rate_along_study(df, samples, study)

### Save columns to a csv file

In [ ]:
utils.columns_to_csv(df=df,
                   samples=samples,
                   columns=['sample', 'construct','full_sequence','roi_sequence','mut_bases','info_bases'],
                   title=f"seq_and_reactivity_{study}",
                   path='data/figs/date/{study}')

### Save construct vs deltaG 

In [ ]:
utils.deltaG_vs_construct_to_csv(df=df,    
                                 title=f"deltaG_vs_construct.csv", 
                                 path = f"data/figs/date/{study}", 
                                 samples=samples)